## 두 번째 페이지 이후의 제품 크롤링

In [1]:
# 크롤링에 필요한 패키지 불러오기.
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [2]:
# 1. 크롬 브라우저 자동 설정
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

[코드 설명]
- `webdriver.ChromeOptions()`: 크롬 드라이버 실행 옵션 설정 객체. (창 크기 조절 등)
- `.add_argument('--start-maximized')`: options 객체에 실행 시 브라우저를 최대화 상태로 시작하라는 인자 추가.
    - 다른 인자)
        - `--headless` : 브라우저 창 없이 백그라운드로 실행
        - `--disable-popup-blocking` : 팝업 차단 해제
        - `--incognito` : 시크릿 모드로 실행
        - `--window-size=1920,1080`: 브라우저 창 크기 지정
- `webdriver.Chrome(...)`: 실제로 **크롬 브라우저 인스턴스를 실행**하는 메서드.
    - `service=Service(...)`: ChromeDriver를 직접 설치하지 않아도 자동으로 설치하고 실행하게 해주는 도우미.
    - `options=options`: 위에서 만든 options 객체를 적용해 브라우저 실행 방식을 조절.
- `ChromeDriverManager().install()`: 자동으로 최신 드라이버 설치.

In [3]:
# 2. 수집할 URL로 이동
url = "https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100020006"
driver.get(url)
time.sleep(5)   # 페이지 로딩 대기

[코드 설명]
- driver.get(url): 해당 URL로 브라우저 이동.
- 동적 로딩이 많아 time.sleep()으로 로딩을 기다리는 방식 필요, 추후 WebDriverWait으로 개선하면 더 안정적.

In [4]:
# 3. 제품 페이지로 이동 (예시로 첫 번째 제품 클릭)
product_links = driver.find_elements(By.CSS_SELECTOR, '.prd_info .tx_name')
product_links[1].click()   # 인덱스 번호 바꿔가면서 여러 제품 크롤링.
time.sleep(5)

[코드 설명]
- `.find_elements()`: 여러 요소를 찾아 리스트로 반환.
    - 위의 코드의 목적은 **상품 이름을 담고 있는 a 태그(=제품 상세 링크)**를 모두 찾는 것.
    - `'.prd_info .tx_name'` 은 HTML 문서 내 < div > 태그의 클래스명. HTML안의 클래스를 CSS에서 선택할 때는 반드시 '.'를 붙여야 함.
    - 즉, 위의 코드는 클래스명이 .prd_info
- CSS 선택자 문법
    - `.className`: 클래스 선택
    - `#idName`: ID 선택
    - `tagname`: 태그 선택
    - `div.className`: 특정 태그+클래스 (ex: div.prd_info = < div class="prd_info" >)
    - `.class1 .class2`: 클래스 중첩 선택. (class1 안에 있는 class2의 요소)

In [5]:
# 4. 리뷰 탭으로 이동
driver.switch_to.window(driver.window_handles[-1])
review_tab = driver.find_element(By.CSS_SELECTOR, 'a.goods_reputation')
review_tab.click()
time.sleep(5)

[코드 설명]
- `href="javascript:;`: 링크가 실제 페이지 이동이 아니라 JavaScript로 이벤트 트리거함. 따라서 Selenium으로 `.click()` 해야만 리뷰 내용이 로딩됨.

In [6]:
# 5. 리뷰 수집

# 제품명 가져오기
product_name = driver.find_element(By.CSS_SELECTOR, 'p.prd_name').text.strip()

# 리뷰 탭 클릭
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementNotInteractableException   # 예외처리(except)를 위해 import.
try:
    # 리뷰 탭이 클릭 가능한 상태가 될 때 까지 대기기
    review_tab = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.goods_reputation'))
    )
    driver.execute_script("arguments[0].click();", review_tab)   # JS 클릭으로 안정성 ↑

    # 리뷰 목록이 나타날 때 까지 대기
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'ul#gdasList li'))
    )
    print("✅ 리뷰 탭 클릭 및 로딩 완료")
except TimeoutException:
    print("❌ 리뷰 탭 또는 리뷰 목록 로딩 실패")
    driver.quit()

# 리뷰 수집 시작
data = []
for i in range(98):   # (리뷰 페이지 수 - 2) 만큼 반복
    review_items = driver.find_elements(By.CSS_SELECTOR, 'ul#gdasList li')   # 각 리뷰의 전체 li 태그 기준
    print (f"[Page {i+1}] 리뷰 개수:", len(review_items))

    # ✅ 여기 바로 아래에 추가!
    print(f"[Page {i+1}] 리뷰 개수 (전체):", len(review_items))
    for idx, review in enumerate(review_items):
        print(f"  ┗ 리뷰 {idx+1}: {review.text[:30]}...")  # 앞 30자만 출력해서 가독성 ↑

    for review in review_items:
        try:
            # ID
            user_id = review.find_element(By.CSS_SELECTOR, '.info_user .id').text.strip()
            
            # 스킨타입
            skin_type = review.find_element(By.CSS_SELECTOR, '.tag span').text.strip()  # 젤 앞의 피부타입만 수집됨. (피부톤은 수집 X.)
            
            # 별점 ★★★★☆
            try:
                star_style = review.find_element(By.CSS_SELECTOR, '.score_area .review_point .point').get_attribute('style')   # style 속성을 가져옴. (별 개수에 따라 style의 퍼센트가 다름)
                rating = int(int(star_style.split(':')[1].replace('%;', '').strip()) / 20)   # 20% = 별 1개
            except:
                rating = 0   # 별점이 없는 경우 예외처리

            # 날짜
            date = review.find_element(By.CSS_SELECTOR, '.date').text.strip()

#            # 평가요소(발색력, 지속력 등등) -> 없을수도 있으므로 예외처리.
#            tag_elements = review.find_elements(By.CSS_SELECTOR, '.poll_sample')
#            tags = [tag.text.strip() for tag in tag_elements]
#            tags_joined = ", ".join(tags)

            # 리뷰 본문
            try:
                review_text = review.find_element(By.CSS_SELECTOR, '.txt_inner').text.strip()
            except:
                review_text = ""

            data.append({
                'Product': product_name,   # 제품명 추가
                'ID': user_id,
                'SkinType': skin_type,
                'Rating': rating,
                'Date': date,
#                'Tags': tags_joined,
                'Review': review_text
            })
        except Exception as e:
            print("⚠️ 리뷰 수집 중 오류:", e)
            continue
    
    # 다음 페이지 클릭
    try:
        paging_area = driver.find_element(By.CLASS_NAME, 'pageing')
        next_page = paging_area.find_element(By.CSS_SELECTOR, f'a[data-page-no="{i+2}"]')
        driver.execute_script("arguments[0].click();", next_page)
        time.sleep(2)

        # 리뷰 리스트 로딩 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'ul#gdasList li'))
        )
    except:
        print(f"⛔ 다음 페이지 {i+2} 없음 or 버튼 클릭 실패:", e)
        break

# 종료
driver.quit()

✅ 리뷰 탭 클릭 및 로딩 완료
[Page 1] 리뷰 개수: 33
[Page 1] 리뷰 개수 (전체): 33
  ┗ 리뷰 1: 하2보
TOP 283
복합성 여름쿨톤 모공
5점만점에 ...
  ┗ 리뷰 2: ontheload
복합성 여름쿨톤 모공 미백
5점만점에...
  ┗ 리뷰 3: ...
  ┗ 리뷰 4: 아따아따
TOP 170
지성 여름쿨톤 모공 트러블
한달...
  ┗ 리뷰 5: ...
  ┗ 리뷰 6: ...
  ┗ 리뷰 7: ...
  ┗ 리뷰 8: naayeeoon
민감성 봄웜톤 각질 다크서클
5점만점...
  ┗ 리뷰 9: ...
  ┗ 리뷰 10: ...
  ┗ 리뷰 11: 자칭소비요청
TOP 760
건성 여름쿨톤 모공 미백
5...
  ┗ 리뷰 12: ...
  ┗ 리뷰 13: ...
  ┗ 리뷰 14: ...
  ┗ 리뷰 15: ...
  ┗ 리뷰 16: 한꼬라지
TOP 597
건성 쿨톤 민감성 아토피
5점만...
  ┗ 리뷰 17: ...
  ┗ 리뷰 18: ...
  ┗ 리뷰 19: ...
  ┗ 리뷰 20: 스틱콘버터
TOP 51
복합성 봄웜톤 모공 미백
한달이...
  ┗ 리뷰 21: ...
  ┗ 리뷰 22: ...
  ┗ 리뷰 23: ...
  ┗ 리뷰 24: 쏭s메틱
건성 봄웜톤 민감성 블랙헤드
5점만점에 5점
...
  ┗ 리뷰 25: ...
  ┗ 리뷰 26: ...
  ┗ 리뷰 27: 사라이
TOP 894
복합성 여름쿨톤 민감성 잡티
5점...
  ┗ 리뷰 28: ...
  ┗ 리뷰 29: ...
  ┗ 리뷰 30: ...
  ┗ 리뷰 31: ...
  ┗ 리뷰 32: 햄서터
지성 여름쿨톤 모공 블랙헤드
5점만점에 5점
2...
  ┗ 리뷰 33: ...
⚠️ 리뷰 수집 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":".info_user .id"}
  (Session info: chrome=134.0.69

[코드 설명]
- `.find_element()` → 요소 하나 찾음 (없으면 오류 발생)
- `.find_elements()` → 리스트로 반환 (없으면 빈 리스트)

In [8]:
# 6. 저장
df = pd.DataFrame(data)
df.to_csv("oliveyoung_reviews.csv", index=False, encoding='utf-8-sig', mode='a', header=False)
print("✅ CSV 저장")
driver.quit()

✅ CSV 저장


[코드 설명]
- #3에서 인덱스 번호를 바꿔가면서 csv파일을 저장할 경우 기존 파일에 새로운 데이터들이 덮어씌워지는 문제 발생.
- 따라서 제품명 별로 구별하여 csv파일 생성.

또 다른 방법: csv저장 모드를 `append` 모드로 바꾸기
``` python
df.to_csv("oliveyoung_reviews.csv", index=False, encoding='utf-8=sig', mode='a', header=False)
```
- 이 경우 첫 번째 제품을 수집할 때만 `header=True`로 해야 CSV에 컬럼명이 들어감.
- 이후 제품들에는 `header=False`로 해야 컬럼명이 중복 저장되지 않음.